# DFA AES Attack

In [1]:
import chipwhisperer as cw

scope = cw.scope()
scope.gain.db = 25
scope.adc.samples = 129
scope.adc.offset = 0
scope.adc.basic_mode = "rising_edge"
scope.clock.clkgen_freq = 10E6
scope.clock.adc_src = "clkgen_x4"
scope.trigger.triggers = "tio4"
scope.io.tio1 = "serial_rx"
scope.io.tio2 = "serial_tx"
scope.io.hs2 = "clkgen"

In [2]:
target = cw.target(scope, cw.targets.CW305, fpga_id='35t', force=False)
# target = cw.target(scope, cw.targets.CW305, fpga_id='100t', force=True)

 **make sure the DIP switches on the CW305 board are set as follows**:
- J16 = 1 `route/drive the Artix target from chipwhisperer-lite board clock`
- K16 = 0 `disable return clock from PLLs being outputed to the ChipWhisperer's HS-In channel`

In [3]:
target.vccint_set(1.0)
# disabe all PLLs
target.pll.pll_enable_set(False)
target.pll.pll_outenable_set(False, 0)
target.pll.pll_outenable_set(False, 1)
target.pll.pll_outenable_set(False, 2)


# 1ms is plenty of idling time
target.clkusbautooff = False
target.clksleeptime = 1

In [4]:
# ensure ADC is locked:
scope.clock.reset_adc()
assert (scope.clock.adc_locked), "ADC failed to lock"

In [5]:
target.usb_clk_setenabled(1)

In [6]:
#Basic setup
scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "clock_xor" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.glitch.width = 10
scope.io.hs2 = "glitch"  # output glitch_out on the clock line
scope.io.glitch_lp = False
print(scope.glitch)

clk_src     = clkgen
width       = 10.15625
width_fine  = 0
offset      = 10.15625
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 0
repeat      = 1
output      = clock_xor



In [7]:
import chipwhisperer.common.results.glitch as glitch
#gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset"])
gc = glitch.GlitchController(groups=["column0", "column1", "column2", "column3", "other", \
                                     "reset", "normal"], parameters=["width", "offset", "ext_offset", "N"])

gc.set_range("width", 1, 2)
gc.set_range("offset", -6, -5)
gc.set_range("ext_offset", 0, 10)
gc.set_range("N", -5, 5)
gc.set_global_step([0.4])

gc.display_stats()

IntText(value=0, description='column0 count:', disabled=True)

IntText(value=0, description='column1 count:', disabled=True)

IntText(value=0, description='column2 count:', disabled=True)

IntText(value=0, description='column3 count:', disabled=True)

IntText(value=0, description='other count:', disabled=True)

...

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=1.0, continuous_update=False, description='width setting:', disabled=True, max=2.0, min=1.0,…

FloatSlider(value=-6.0, continuous_update=False, description='offset setting:', disabled=True, max=-5.0, min=-…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

FloatSlider(value=-5.0, continuous_update=False, description='N setting:', disabled=True, max=5.0, min=-5.0, r…

In [8]:
print(scope.adc.trig_count)

3633288


In [9]:
scope.glitch.repeat = 1

In [10]:
scope.glitch.output = "clock_xor"

In [11]:
target.vccint_set(1.00)

In [12]:
outputs = []
results = []
obf = []

In [13]:
scope.clock.clkgen_freq = 10E6

In [14]:
from Crypto.Cipher import AES

ktp = cw.ktp.Basic()
key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
# initialize cipher to verify DUT result:
cipher = AES.new(bytes(key), AES.MODE_ECB)

goldciph = list(cipher.encrypt(bytes(text)))

In [15]:
from tqdm import tnrange
import numpy as np
import time
from Crypto.Cipher import AES


project_file = "projects/Tutorial_HW_CW305_DFA.cwp"
project = cw.create_project(project_file, overwrite=True)
traces = []
textin = []
keys = []


def check_column_glitch(glitched_ct, gold_ct, column):
    column_lookup = [[0, 13, 10, 7], [4, 1, 14, 11], [8, 5, 2, 15], [12, 9, 6, 3]] #shift rows
    for byte in column_lookup[column]:
        if glitched_ct[byte] == gold_ct[byte]:
            return False
    return True
scope.adc.timeout = 0.1

class DelayedKeyboardInterrupt:
    def __enter__(self):
        self.signal_received = False
        self.old_handler = signal.signal(signal.SIGINT, self.handler)

    def handler(self, sig, frame):
        self.signal_received = (sig, frame)
        logging.debug('SIGINT received. Delaying KeyboardInterrupt.')

    def __exit__(self, type, value, traceback):
        signal.signal(signal.SIGINT, self.old_handler)
        if self.signal_received:
            self.old_handler(*self.signal_received)
    
import logging, signal
logging.getLogger().setLevel(logging.ERROR)
for glitch_setting in gc.glitch_values():
    with DelayedKeyboardInterrupt():
        # run aux stuff that should come before trace here
        scope.glitch.offset = glitch_setting[1]
        scope.glitch.width = glitch_setting[0]
        scope.glitch.ext_offset = glitch_setting[2]
        scope.glitch.offset_fine = glitch_setting[3] / 0.4

        ret = cw.capture_trace(scope, target, text, key)
        if not ret:
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))
            continue

        if (list(ret.textout) == goldciph):
            gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))
            continue
        else:
            pass
            #print(list(ret.textout))
            #print(goldciph)
            #print(scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset)
        #trace += scope.getLastTrace()
        #check for a glitch in each column of AES
        traces.append(ret.wave)
        project.traces.append(ret)
        outputs.append(list(ret.textout))
        column_glitches = []
        for column in range(4):
            if check_column_glitch(ret.textout, goldciph, column):
                column_glitches.append(column)
        #We're looking for single column glitches here
        if len(column_glitches) == 1:
            gc.add("column{}".format(column_glitches[0]), (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))
            obf.append(ret.textout)
            print(scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3], column_glitches[0])
            
        else:
            gc.add("other", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset, glitch_setting[3]))

1.171875 -5.859375 3 4.6000000000000005 1
1.171875 -5.859375 3 4.2 1
1.171875 -5.46875 3 -3.3999999999999995 1
1.171875 -5.46875 3 -1.4 1
1.171875 -5.46875 3 -0.5999999999999999 1
1.171875 -5.46875 3 -4.6 1
1.171875 -5.46875 3 -3.7999999999999994 1
1.171875 -5.46875 3 -2.9999999999999996 1
1.171875 -5.46875 3 -0.19999999999999984 1
1.171875 -5.46875 3 -4.6 1
1.171875 -5.46875 3 -2.1999999999999997 1
1.171875 -5.46875 3 0.20000000000000018 1
1.171875 -5.46875 4 2.6 0
1.171875 -5.46875 4 3.0 0
1.171875 -5.46875 4 4.6000000000000005 0
1.171875 -5.46875 4 3.0 0
1.171875 -5.46875 4 3.4 0
1.171875 -5.46875 4 4.6000000000000005 0
1.171875 -5.078125 4 -4.6 0
1.171875 -5.078125 4 -5 0
1.171875 -5.078125 4 -4.6 0
1.171875 -5.078125 4 -4.199999999999999 0
1.171875 -5.078125 4 -3.7999999999999994 0
1.171875 -5.078125 4 -3.3999999999999995 0
1.171875 -5.078125 4 -2.5999999999999996 0
1.171875 -5.078125 4 -1.7999999999999998 0
1.171875 -5.078125 4 -0.9999999999999999 0
1.171875 -5.078125 4 0.2000000

In [16]:
project.save()

In [17]:
import phoenixAES
r10=phoenixAES.crack_bytes(obf, goldciph, encrypt=True, verbose=2)

22ad61fb53701ce48d8b202b9ba752df: group None
22ad61fb53701ce40b8b202b9ba752df: group None
22ad615053701c4f968b208042a75274: group 1
22ad61fb53701ce48d8b202b9ba752df: group None
22ad61fb53701ce48d8b202b9ba752df: group None
22ad61fb53701ce48d8b202b9ba752df: group None
22ad615053701c4f968b20809ba75274: group None
22ad615053701c4f968b20809ba75274: group None
22ad61fb53701ce40b8b202b9ba752df: group None
22ad615053701c4f968b20809ba75274: group None
22ad615053701c4f968b20809ba75274: group None
22ad615053701c4f968b20809ba75274: group None
0483615050701c0f4148a619afd98663: group None
1483615094701c0f7a48a618afd98672: group None
0483615050701c0f4748a619afd98673: group None
0c836150d0701c0f4948a60927d98663: group None
0483615010701c0f7a48a618afd98673: group None
5c836150d4701c0f52480e1823d98672: group None
0c836150d0701c0f4b48a60927d98663: group None
0c836150d0701c0f4d48a601278b8663: group None
1483615014701c0f7a48a618afd98673: group None
0c836150d0701c0f4b48a61927d98673: group None
0c836150d0701

In [18]:
import phoenixAES
outputs2=phoenixAES.convert_r8faults_bytes(outputs, goldciph, encrypt=True)
r10=phoenixAES.crack_bytes(outputs2, goldciph, encrypt=True, verbose=2)

acb3615015701c1c968b9b5b42c28674: group 0
224e6150e1701c4f968b203942a7a674: group 1
22b3005015321c4ff08b205b42a78620: group 2
22b3611615707c4f963f205b3ca78674: group 3
acb3615015701c1c968b9b5b42c28674: group 0
224e6150e1701c4f968b203942a7a674: group 1
22b3005015321c4ff08b205b42a78620: group 2
22b3611615707c4f963f205b3ca78674: group 3
bab3615015701ccf968bc25b42208674: group 0
2241615009701c4f968b20c842a72174: group 1
22b3565015e51c4f858b205b42a786c7: group 2
22b361c01570f64f96dc205b2ea78674: group 3
20b3615015701cb4968bc45b42fb8674: group 0
22bb61505e701c4f968b205a42a70c74: group 1
22b33f5015821c4f4e8b205b42a786d9: group 2
22b361001570714f9613205b6ca78674: group 3
bab3615015701ccf968bc25b42208674: group 0
2241615009701c4f968b20c842a72174: group 1
22b3565015e51c4f858b205b42a786c7: group 2
22b361c01570f64f96dc205b2ea78674: group 3
bab3615015701ccf968bc25b42208674: group 0
2241615009701c4f968b20c842a72174: group 1
22b3565015e51c4f858b205b42a786c7: group 2
22b361c01570f64f96dc205b2ea78674: 

In [19]:
if r10 is not None:
    from chipwhisperer.analyzer.attacks.models.aes.key_schedule import key_schedule_rounds
    recv_key = key_schedule_rounds(bytearray.fromhex(r10), 10, 0)
    print("AES Key:")
    print(''.join("%02x" % x for x in recv_key))
else:
    print("Sorry, no key found, try another campain, maybe with different parameters...")

AES Key:
2b7e151628aed2a6abf7158809cf4f3c


In [20]:
bytearray(key_schedule_rounds(key, 0, 10))

CWbytearray(b'd0 14 f9 a8 c9 ee 25 89 e1 3f 0c c8 b6 63 0c a6')

In [21]:
bytearray(key)

CWbytearray(b'2b 7e 15 16 28 ae d2 a6 ab f7 15 88 09 cf 4f 3c')

In [22]:
key = list(key)
assert (recv_key == key), "Failed to recover encryption key\nGot:      {}\nExpected: {}".format(recv_key, key)

In [23]:
scope.dis()
target.dis()